In [1]:
import sys
sys.path.append("../../src/models")

In [2]:
import numpy as np
from actions import city_restrictions, costs

In [3]:
from simulate_pandemic import init_infection, spread_infection, lambda_leak_expose, update_population
from simulate_pandemic import main

In [4]:
from numpy.random import default_rng
import pandas as pd
from datetime import datetime
from tqdm import tqdm

In [5]:
from joblib import Parallel, delayed

In [6]:
def make_df(week, sim, action, data):
    df = pd.DataFrame(pd.Series(data[:, 1]).value_counts()).T
    df['simulation'] = sim
    df['week'] = week
    df['action'] = action
    df = df.rename(columns = {
                              -1 : 'removed',
                               0 : 'susceptible',
                               1 : 'exposed',
                               2 : 'infected',
                               3 : 'hospitalized'
                              }
                  )
    
    return df

In [7]:
def simulate_and_save(policy_number, 
                      policy,
                      n_sims,
                      step_size=7,
                      folder='generated_sims/'):
    dfs = []
    for sim in range(1, n_sims+1):
        
        sim_name = f'{policy_number}_{sim}'
                
        data, pop_matrix = main(gpickle_path=gpickle_path,
                                p_r=p_r,
                                policy=policy,
                                disable_tqdm=True,
                                days=step_size*len(policy),
                                step_size=step_size)
        weeks = (step_size/7) * len(policy)
        weekly_data = [data[i*7] for i in range(int(weeks))]
        df = pd.concat([make_df(week, sim_name, action, data)
                             for week, (data, action)
                             in enumerate(zip(weekly_data, policy))])
        #df.to_parquet(folder + f'{sim_name}.parquet')
        dfs += [df]
    return dfs

In [8]:
rng = default_rng(None)

prhome = 0.06
p_r = {
    'home'    :  prhome,
    'neighbor':  .1*prhome,
    'work'    :  .1*prhome,
    'school'  :  .15*prhome,
}

gpickle_path = '../../data/processed/SP_multiGraph_Job_Edu_Level.gpickle'

In [9]:
n_policies = 60000
n_sims = 4

In [10]:
costs_keys = list(costs.keys())
costs_values = np.array(list(costs.values()))
p = (1 - costs_values) / (1 - costs_values).sum()

policy = [rng.choice(costs_keys,
                     size=int(364/14),
                     replace=True, p=p) for i in range(n_policies)]
assert len(set([tuple(p) for p in policy])) == len(policy)

In [11]:
res = Parallel(n_jobs=-1)(delayed(simulate_and_save)(i, pol, n_sims) 
                              for i, pol in tqdm(enumerate(policy),
                                                 total=len(policy)))

  0%|          | 24/60000 [01:19<53:04:24,  3.19s/it]

KeyboardInterrupt: 